In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

/tmp/ipykernel_1586/1799943160.py:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png', 'pdf')


# Exercise 6

<img src='./images/06.png' width=800>

In [ ]:
import os
import mlflow
os.environ['MLFLOW_TRACKING_URI'] = './mlruns'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise_6')

2025/04/07 18:14:00 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_6' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_02/mlruns/688296389628558366', creation_time=1744037040958, experiment_id='688296389628558366', last_update_time=1744037040958, lifecycle_stage='active', name='Exercise_6', tags={}>

In [ ]:
from sklearn.datasets import make_moons
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import optuna
import torch.nn as nn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import (train_network, accuracy_score_wrapper, 
                f1_score_wrapper, roc_auc_score_micro_wrapper, 
                weight_reset, set_seed)
from torchinfo import summary
from mlflow import MlflowClient
from mlflow.types import Schema, TensorSpec
from mlflow.models import ModelSignature


random_state = 42
set_seed(random_state)

X_train, Y_train = make_moons(n_samples=8000, noise=0.4, random_state=random_state)
X_valid, Y_valid = make_moons(n_samples=200, noise=0.4, random_state=random_state)

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                            torch.tensor(Y_train, dtype=torch.long))
valid_dataset = TensorDataset(torch.tensor(X_valid, dtype=torch.float32),
                            torch.tensor(Y_valid, dtype=torch.long))


def plot_results(data_df, close=True):
    sns.lineplot(data_df, x='epoch', y='valid AUC', label='valid AUC')
    plt.xlabel('epoch')
    plt.ylabel('valid AUC')
    plt.title('valid AUC')
    fig = plt.gcf()
    if close:
        plt.close()
    return fig

def plot_results_hidden_neurons(data_list, close=True):
    sns.lineplot(x=range(100, 100*(len(data_list)+1), 100), y=data_list)
    plt.xlabel('hidden neurons')
    plt.ylabel('valid AUC')
    plt.title('valid AUC per hidden neurons')
    plt.xticks(range(100, 100*(len(data_list)+1), 100))
    fig = plt.gcf()
    if close:
        plt.close()
    return fig

run_id = '219aed50bb7f4f8cbe70429ade80c1a5' # best valid auc from exercise 2

client = MlflowClient()
run_data = client.get_run(run_id).data

params = run_data.params
epochs = 20
in_features = 2
out_features = 2
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
params['device'] = device
params['epochs'] = epochs
loss_func = nn.CrossEntropyLoss()
activation_functions = {
'ReLU': nn.ReLU(),
'Tanh': nn.Tanh(),
'LeakyReLU': nn.LeakyReLU(),
'Sigmoid': nn.Sigmoid()
}

train_dataloader = DataLoader(train_dataset, batch_size=int(params['batch_size']),shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=int(params['batch_size']))

results_neurons = []
neurons = 1000
for neuron in range(100, neurons, 100):
    print(f'trainig: {neuron} neurons')
    params['hidden_neurons'] = neuron
    sequential_layer = [
    nn.Linear(in_features, neuron),
    activation_functions[params['activation']]
    ]
    for _ in range(int(params['layers'])):
        sequential_layer.append(nn.Linear(neuron, neuron))
        sequential_layer.append(activation_functions[params['activation']])
    sequential_layer.append(nn.Linear(neuron, out_features))

    model = nn.Sequential(*sequential_layer)
    optimizer = torch.optim.SGD(model.parameters(), lr=float(params['learning_rate']))

    with mlflow.start_run(nested=True, run_name=f'neurons_{neuron}'):
        mlflow.log_params(params)

        
        with open ("model_summary.txt", "w") as f:
            f.write(str(summary(model)))
        mlflow.log_artifact("model_summary.txt")
        # model.apply(weight_reset)
        fc_results = train_network(
            model=model,
            loss_func=loss_func,
            train_loader=train_dataloader,
            valid_loader=valid_dataloader,
            epochs=epochs,
            optimizer=optimizer,
            score_funcs={'Acc':accuracy_score_wrapper, 'F1':f1_score_wrapper, 'AUC':roc_auc_score_micro_wrapper },
            device=device,
            # checkpoint_file_save='model.pth',
            # checkpoint_every_x=10,
        )
        
        # input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
        # output_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
        # signature = ModelSignature(inputs=input_schema, outputs=output_schema)
        # mlflow.pytorch.log_model(model, "model", signature=signature)
        mlflow.log_figure(plot_results(fc_results), "valid_AUC.png")
    results_neurons.append(fc_results['valid AUC'].iloc[-1])
mlflow.log_figure(plot_results_hidden_neurons(results_neurons), 'valid_auc_per_hidden_neurons.png')

trainig: 100 neurons


Epoch: 100%|██████████| 20/20 [00:07<00:00,  2.51it/s]


trainig: 200 neurons


Epoch: 100%|██████████| 20/20 [00:09<00:00,  2.08it/s]


trainig: 300 neurons


...

Epoch: 100%|██████████| 20/20 [00:48<00:00,  2.45s/it]


trainig: 800 neurons


Epoch: 100%|██████████| 20/20 [00:54<00:00,  2.73s/it]


trainig: 900 neurons


Epoch: 100%|██████████| 20/20 [01:12<00:00,  3.62s/it]


<img src='./images/valid_auc_hidden_neurons.png' width=800>

<img src='./images/valid_auc_per_hidden_neurons.png' width=800>